# Aggregating Padas DataFrames

## Performing database-style operations on DataFrames 

In [1]:
import pandas as pd

weather = pd.read_csv('data/nyc_weather_2018.csv')
weather.head()

date datatype            station attributes  value
0  2018-01-01T00:00:00     PRCP  GHCND:US1CTFR0039       ,,N,    0.0
1  2018-01-01T00:00:00     PRCP  GHCND:US1NJBG0015       ,,N,    0.0
2  2018-01-01T00:00:00     SNOW  GHCND:US1NJBG0015       ,,N,    0.0
3  2018-01-01T00:00:00     PRCP  GHCND:US1NJBG0017       ,,N,    0.0
4  2018-01-01T00:00:00     SNOW  GHCND:US1NJBG0017       ,,N,    0.0

In [2]:
weather.dtypes

date           object
datatype       object
station        object
attributes     object
value         float64
dtype: object

### Querying DataFrames

In [3]:
snow_data = weather.query('datatype == "SNOW" and value > 0 and station.str.contains("US1NY")', engine='python')
snow_data.head()

date datatype            station attributes  value
114  2018-01-01T00:00:00     SNOW  GHCND:US1NYWC0019       ,,N,   25.0
789  2018-01-04T00:00:00     SNOW  GHCND:US1NYNS0007       ,,N,   41.0
794  2018-01-04T00:00:00     SNOW  GHCND:US1NYNS0018       ,,N,   10.0
798  2018-01-04T00:00:00     SNOW  GHCND:US1NYNS0024       ,,N,   89.0
800  2018-01-04T00:00:00     SNOW  GHCND:US1NYNS0030       ,,N,  102.0

This is equivalent to querying the `weather.db` SQLite database for 

```sql
SELECT * 
FROM weather 
WHERE datatype == "SNOW" AND value > 0 AND station LIKE "%US1NY%"
```

In [4]:
import sqlite3

with sqlite3.connect('data/weather.db') as connection:
    snow_data_from_db = pd.read_sql(
        'SELECT * FROM weather WHERE datatype == "SNOW" and value > 0 and station LIKE "%US1NY%"',
        connection)
snow_data_from_db.head()

date datatype            station attributes  value
0  2018-01-01T00:00:00     SNOW  GHCND:US1NYWC0019       ,,N,   25.0
1  2018-01-04T00:00:00     SNOW  GHCND:US1NYNS0007       ,,N,   41.0
2  2018-01-04T00:00:00     SNOW  GHCND:US1NYNS0018       ,,N,   10.0
3  2018-01-04T00:00:00     SNOW  GHCND:US1NYNS0024       ,,N,   89.0
4  2018-01-04T00:00:00     SNOW  GHCND:US1NYNS0030       ,,N,  102.0

In [5]:
snow_data.reset_index().drop(columns='index').equals(snow_data_from_db)

True

### Merging DataFrames

Merging dataframes deals with how to line them up by row. When referring to databases, merging is traditionally called a join.

In [6]:
station_info = pd.read_csv('data/weather_stations.csv')
station_info.head()

id                           name   latitude  longitude  \
0  GHCND:US1CTFR0022        STAMFORD 2.6 SSW, CT US  41.064100 -73.577000   
1  GHCND:US1CTFR0039          STAMFORD 4.2 S, CT US  41.037788 -73.568176   
2  GHCND:US1NJBG0001      BERGENFIELD 0.3 SW, NJ US  40.921298 -74.001983   
3  GHCND:US1NJBG0002  SADDLE BROOK TWP 0.6 E, NJ US  40.902694 -74.083358   
4  GHCND:US1NJBG0003           TENAFLY 1.3 W, NJ US  40.914670 -73.977500   

   elevation  
0       36.6  
1        6.4  
2       20.1  
3       16.8  
4       21.6

In [7]:
station_info.id.describe()

count                   279
unique                  279
top       GHCND:US1CTFR0022
freq                      1
Name: id, dtype: object

In [8]:
weather.station.describe()

count                 78780
unique                  110
top       GHCND:USW00094789
freq                   4270
Name: station, dtype: object

In [9]:
def get_row_count(*dfs):
    return [df.shape[0] for df in dfs]
get_row_count(station_info, weather)

[279, 78780]

By default, `merge()` performs an inner join. We simply specify the columns to use for the join. The left dataframe is the one we call `merge()` on, and the right one is passed in as an argument:

In [10]:
inner_join = weather.merge(station_info, left_on='station', right_on='id')
inner_join.sample(5, random_state=0)

date datatype            station attributes  value  \
10739  2018-08-07T00:00:00     SNOW  GHCND:US1NJMN0069       ,,N,    0.0   
45188  2018-12-21T00:00:00     TMAX  GHCND:USW00014732   ,,W,2400   16.7   
59823  2018-01-15T00:00:00     WDF5  GHCND:USW00094741       ,,W,   40.0   
10852  2018-10-31T00:00:00     PRCP  GHCND:US1NJMN0069      T,,N,    0.0   
46755  2018-05-05T00:00:00     SNOW  GHCND:USW00014734       ,,W,    0.0   

                      id                                         name  \
10739  GHCND:US1NJMN0069                   LONG BRANCH 1.7 SSW, NJ US   
45188  GHCND:USW00014732                     LAGUARDIA AIRPORT, NY US   
59823  GHCND:USW00094741                     TETERBORO AIRPORT, NJ US   
10852  GHCND:US1NJMN0069                   LONG BRANCH 1.7 SSW, NJ US   
46755  GHCND:USW00014734  NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US   

        latitude  longitude  elevation  
10739  40.275368 -74.006027        9.4  
45188  40.779440 -73.880350        3.4  
59823  40.850000 -74.061390        2.7  
10852  40.275368 -74.006027        9.4  
46755  40.682500 -74.169400        2.1

We can remove the duplication of information in the `station` and `id` columns by renaming one of them before the merge and then simply using `on`:

In [11]:
weather.merge(
    station_info.rename(dict(id='station'), axis=1),
    on='station').sample(5, random_state=0)

date datatype            station attributes  value  \
10739  2018-08-07T00:00:00     SNOW  GHCND:US1NJMN0069       ,,N,    0.0   
45188  2018-12-21T00:00:00     TMAX  GHCND:USW00014732   ,,W,2400   16.7   
59823  2018-01-15T00:00:00     WDF5  GHCND:USW00094741       ,,W,   40.0   
10852  2018-10-31T00:00:00     PRCP  GHCND:US1NJMN0069      T,,N,    0.0   
46755  2018-05-05T00:00:00     SNOW  GHCND:USW00014734       ,,W,    0.0   

                                              name   latitude  longitude  \
10739                   LONG BRANCH 1.7 SSW, NJ US  40.275368 -74.006027   
45188                     LAGUARDIA AIRPORT, NY US  40.779440 -73.880350   
59823                     TETERBORO AIRPORT, NJ US  40.850000 -74.061390   
10852                   LONG BRANCH 1.7 SSW, NJ US  40.275368 -74.006027   
46755  NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US  40.682500 -74.169400   

       elevation  
10739        9.4  
45188        3.4  
59823        2.7  
10852        9.4  
46755        2.1

In [12]:
left_join = station_info.merge(
    weather, left_on='id', right_on='station', how='left')
right_join = weather.merge(
    station_info, left_on='station', right_on='id', how='right')

In [13]:
right_join[right_join.datatype.isna()].head()

date datatype station attributes  value                 id  \
0    NaN      NaN     NaN        NaN    NaN  GHCND:US1CTFR0022   
344  NaN      NaN     NaN        NaN    NaN  GHCND:US1NJBG0001   
345  NaN      NaN     NaN        NaN    NaN  GHCND:US1NJBG0002   
718  NaN      NaN     NaN        NaN    NaN  GHCND:US1NJBG0005   
719  NaN      NaN     NaN        NaN    NaN  GHCND:US1NJBG0006   

                              name   latitude  longitude  elevation  
0          STAMFORD 2.6 SSW, CT US  41.064100 -73.577000       36.6  
344      BERGENFIELD 0.3 SW, NJ US  40.921298 -74.001983       20.1  
345  SADDLE BROOK TWP 0.6 E, NJ US  40.902694 -74.083358       16.8  
718        WESTWOOD 0.8 ESE, NJ US  40.983041 -74.015858       15.8  
719            RAMSEY 0.6 E, NJ US  41.058611 -74.134068      112.2

The left and right join as we performed above are equivalent because the side for which we kept the rows without matches was the same in both cases:

In [14]:
left_join.sort_index(axis=1
        ).sort_values(['date', 'station'], ignore_index=True
        ).equals(right_join.sort_index(axis=1
                    ).sort_values(['date', 'station'], ignore_index=True
        ))

True

In [15]:
get_row_count(inner_join, left_join, right_join)

[78780, 78949, 78949]

In [16]:
# a full outer join will keep all the values, regardless of whether or not they exist in both dataframes
# we pass an in indicator=True to add an addititonal column to the resulting dataframe
outer_join = weather.merge(
    station_info[station_info.id.str.contains('US1NY')],
    left_on='station', right_on='id',
    how='outer', indicator=True)

In [17]:
outer_join.head()

date datatype            station attributes  value   id  \
0  2018-01-01T00:00:00     PRCP  GHCND:US1CTFR0039       ,,N,    0.0  NaN   
1  2018-01-02T00:00:00     PRCP  GHCND:US1CTFR0039       ,,N,    0.0  NaN   
2  2018-01-03T00:00:00     PRCP  GHCND:US1CTFR0039       ,,N,    0.0  NaN   
3  2018-01-05T00:00:00     DAPR  GHCND:US1CTFR0039       ,,N,    2.0  NaN   
4  2018-01-05T00:00:00     MDPR  GHCND:US1CTFR0039       ,,N,   15.5  NaN   

  name  latitude  longitude  elevation     _merge  
0  NaN       NaN        NaN        NaN  left_only  
1  NaN       NaN        NaN        NaN  left_only  
2  NaN       NaN        NaN        NaN  left_only  
3  NaN       NaN        NaN        NaN  left_only  
4  NaN       NaN        NaN        NaN  left_only

In [18]:
# view effect of outer join
pd.concat([
    outer_join.query(f'_merge == "{kind}"').sample(2, random_state=0)
    for kind in outer_join._merge.unique()
]).sort_index()

date datatype            station attributes  value  \
23634  2018-04-12T00:00:00     PRCP  GHCND:US1NYNS0043       ,,N,    0.0   
25742  2018-03-25T00:00:00     PRCP  GHCND:US1NYSF0061       ,,N,    0.0   
60645  2018-04-16T00:00:00     TMIN  GHCND:USW00094741       ,,W,    3.9   
70764  2018-03-23T00:00:00     SNWD  GHCND:US1NJHD0002       ,,N,  203.0   
78790                  NaN      NaN                NaN        NaN    NaN   
78800                  NaN      NaN                NaN        NaN    NaN   

                      id                         name   latitude  longitude  \
23634  GHCND:US1NYNS0043     PLAINVIEW 0.4 ENE, NY US  40.785919 -73.466873   
25742  GHCND:US1NYSF0061     CENTERPORT 0.9 SW, NY US  40.891689 -73.383133   
60645                NaN                          NaN        NaN        NaN   
70764                NaN                          NaN        NaN        NaN   
78790  GHCND:US1NYQN0033  HOWARD BEACH 0.4 NNW, NY US  40.662099 -73.841345   
78800  GHCND:US1NYWC0009    NEW ROCHELLE 1.3 S, NY US  40.904000 -73.777000   

       elevation      _merge  
23634       56.7        both  
25742       53.6        both  
60645        NaN   left_only  
70764        NaN   left_only  
78790        2.1  right_only  
78800       21.9  right_only

In [19]:
dirty_data = pd.read_csv('data/dirty_data.csv', index_col='date'
            ).drop_duplicates().drop(columns='SNWD')
dirty_data.head()

station  PRCP   SNOW    TMAX  TMIN  TOBS  WESF  \
date                                                                            
2018-01-01T00:00:00                  ?   0.0    0.0  5505.0 -40.0   NaN   NaN   
2018-01-02T00:00:00  GHCND:USC00280907   0.0    0.0    -8.3 -16.1 -12.2   NaN   
2018-01-03T00:00:00  GHCND:USC00280907   0.0    0.0    -4.4 -13.9 -13.3   NaN   
2018-01-04T00:00:00                  ?  20.6  229.0  5505.0 -40.0   NaN  19.3   
2018-01-05T00:00:00                  ?   0.3    NaN  5505.0 -40.0   NaN   NaN   

                    inclement_weather  
date                                   
2018-01-01T00:00:00               NaN  
2018-01-02T00:00:00             False  
2018-01-03T00:00:00             False  
2018-01-04T00:00:00              True  
2018-01-05T00:00:00               NaN

In [20]:
valid_station = dirty_data.query('station != "?"'
                ).drop(columns=['WESF', 'station'])
station_with_wesf = dirty_data.query('station == "?"').drop(columns=['station', 'TOBS', 'TMIN', 'TMAX'])

Our column for the join is the index in both dataframes, so we must specify `left_index` and `right_index`:

In [21]:
valid_station.merge(
    station_with_wesf, how='left',
    left_index=True, right_index=True
).query('WESF > 0').head()

PRCP_x  SNOW_x  TMAX  TMIN  TOBS inclement_weather_x  \
date                                                                        
2018-01-30T00:00:00     0.0     0.0   6.7  -1.7  -0.6               False   
2018-03-08T00:00:00    48.8     NaN   1.1  -0.6   1.1               False   
2018-03-13T00:00:00     4.1    51.0   5.6  -3.9   0.0                True   
2018-03-21T00:00:00     0.0     0.0   2.8  -2.8   0.6               False   
2018-04-02T00:00:00     9.1   127.0  12.8  -1.1  -1.1                True   

                     PRCP_y  SNOW_y  WESF inclement_weather_y  
date                                                           
2018-01-30T00:00:00     1.5    13.0   1.8                True  
2018-03-08T00:00:00    28.4     NaN  28.7                 NaN  
2018-03-13T00:00:00     3.0    13.0   3.0                True  
2018-03-21T00:00:00     6.6   114.0   8.6                True  
2018-04-02T00:00:00    14.0   152.0  15.2                True

The columns that existed in both dataframes, but didn't form part of the join got suffixes added to their names: `_x` for columns from the left dataframe and `_y` for columns from the right dataframe. We can customize this with the `suffixes` argument:

In [22]:
valid_station.merge(
    station_with_wesf, how='left',
    left_index=True, right_index=True,
    suffixes=('', '_?')
).query('WESF > 0').head()

PRCP   SNOW  TMAX  TMIN  TOBS inclement_weather  PRCP_?  \
date                                                                           
2018-01-30T00:00:00   0.0    0.0   6.7  -1.7  -0.6             False     1.5   
2018-03-08T00:00:00  48.8    NaN   1.1  -0.6   1.1             False    28.4   
2018-03-13T00:00:00   4.1   51.0   5.6  -3.9   0.0              True     3.0   
2018-03-21T00:00:00   0.0    0.0   2.8  -2.8   0.6             False     6.6   
2018-04-02T00:00:00   9.1  127.0  12.8  -1.1  -1.1              True    14.0   

                     SNOW_?  WESF inclement_weather_?  
date                                                   
2018-01-30T00:00:00    13.0   1.8                True  
2018-03-08T00:00:00     NaN  28.7                 NaN  
2018-03-13T00:00:00    13.0   3.0                True  
2018-03-21T00:00:00   114.0   8.6                True  
2018-04-02T00:00:00   152.0  15.2                True

When we are joining on the index, an easier way to do this is to use the `join()` method instead of `merge()`.

In [23]:
valid_station.join(
    station_with_wesf, how='left', rsuffix='_?'
).query('WESF > 0').head()

PRCP   SNOW  TMAX  TMIN  TOBS inclement_weather  PRCP_?  \
date                                                                           
2018-01-30T00:00:00   0.0    0.0   6.7  -1.7  -0.6             False     1.5   
2018-03-08T00:00:00  48.8    NaN   1.1  -0.6   1.1             False    28.4   
2018-03-13T00:00:00   4.1   51.0   5.6  -3.9   0.0              True     3.0   
2018-03-21T00:00:00   0.0    0.0   2.8  -2.8   0.6             False     6.6   
2018-04-02T00:00:00   9.1  127.0  12.8  -1.1  -1.1              True    14.0   

                     SNOW_?  WESF inclement_weather_?  
date                                                   
2018-01-30T00:00:00    13.0   1.8                True  
2018-03-08T00:00:00     NaN  28.7                 NaN  
2018-03-13T00:00:00    13.0   3.0                True  
2018-03-21T00:00:00   114.0   8.6                True  
2018-04-02T00:00:00   152.0  15.2                True

Joins can be very resource-intensive, so it's a good idea to figure out what type of join you need using set operations before trying the join itself. The `pandas` set operations are performed on the index, so whichever columns we will be joining on will need to be the index. Let's go back to the `weather` and `station_info` dataframes and set the station ID columns as the index:

In [24]:
weather.set_index('station', inplace=True)
station_info.set_index('id', inplace=True)

In [25]:
weather.head()

date datatype attributes  value
station                                                          
GHCND:US1CTFR0039  2018-01-01T00:00:00     PRCP       ,,N,    0.0
GHCND:US1NJBG0015  2018-01-01T00:00:00     PRCP       ,,N,    0.0
GHCND:US1NJBG0015  2018-01-01T00:00:00     SNOW       ,,N,    0.0
GHCND:US1NJBG0017  2018-01-01T00:00:00     PRCP       ,,N,    0.0
GHCND:US1NJBG0017  2018-01-01T00:00:00     SNOW       ,,N,    0.0

In [26]:
station_info.head()

name   latitude  longitude  \
id                                                                       
GHCND:US1CTFR0022        STAMFORD 2.6 SSW, CT US  41.064100 -73.577000   
GHCND:US1CTFR0039          STAMFORD 4.2 S, CT US  41.037788 -73.568176   
GHCND:US1NJBG0001      BERGENFIELD 0.3 SW, NJ US  40.921298 -74.001983   
GHCND:US1NJBG0002  SADDLE BROOK TWP 0.6 E, NJ US  40.902694 -74.083358   
GHCND:US1NJBG0003           TENAFLY 1.3 W, NJ US  40.914670 -73.977500   

                   elevation  
id                            
GHCND:US1CTFR0022       36.6  
GHCND:US1CTFR0039        6.4  
GHCND:US1NJBG0001       20.1  
GHCND:US1NJBG0002       16.8  
GHCND:US1NJBG0003       21.6

The intersection will tell us the stations that are present in both dataframes. The result will be the index when performing an inner join:

In [27]:
weather.index.intersection(station_info.index)

Index(['GHCND:US1CTFR0039', 'GHCND:US1NJBG0015', 'GHCND:US1NJBG0017',
       'GHCND:US1NJBG0018', 'GHCND:US1NJBG0023', 'GHCND:US1NJBG0030',
       'GHCND:US1NJBG0039', 'GHCND:US1NJBG0044', 'GHCND:US1NJES0018',
       'GHCND:US1NJES0024',
       ...
       'GHCND:US1NJBG0037', 'GHCND:USC00284987', 'GHCND:US1NJES0031',
       'GHCND:US1NJES0029', 'GHCND:US1NJMD0086', 'GHCND:US1NJMS0097',
       'GHCND:US1NJMN0081', 'GHCND:US1NJMD0088', 'GHCND:US1NJES0040',
       'GHCND:US1NYQN0029'],
      dtype='object', length=110)

In [28]:
weather.index.difference(station_info.index)

Index([], dtype='object')

In [29]:
station_info.index.difference(weather.index)

Index(['GHCND:US1CTFR0022', 'GHCND:US1NJBG0001', 'GHCND:US1NJBG0002',
       'GHCND:US1NJBG0005', 'GHCND:US1NJBG0006', 'GHCND:US1NJBG0008',
       'GHCND:US1NJBG0011', 'GHCND:US1NJBG0012', 'GHCND:US1NJBG0013',
       'GHCND:US1NJBG0020',
       ...
       'GHCND:USC00308322', 'GHCND:USC00308749', 'GHCND:USC00308946',
       'GHCND:USC00309117', 'GHCND:USC00309270', 'GHCND:USC00309400',
       'GHCND:USC00309466', 'GHCND:USC00309576', 'GHCND:USW00014708',
       'GHCND:USW00014786'],
      dtype='object', length=169)

The `union` will show us everything that will be present after a full outer join. Note that we pass in the unique values of the index to make sure we can see the number of stations we will be left with:

In [30]:
weather.index.unique().union(station_info.index)

Index(['GHCND:US1CTFR0022', 'GHCND:US1CTFR0039', 'GHCND:US1NJBG0001',
       'GHCND:US1NJBG0002', 'GHCND:US1NJBG0003', 'GHCND:US1NJBG0005',
       'GHCND:US1NJBG0006', 'GHCND:US1NJBG0008', 'GHCND:US1NJBG0010',
       'GHCND:US1NJBG0011',
       ...
       'GHCND:USW00014708', 'GHCND:USW00014732', 'GHCND:USW00014734',
       'GHCND:USW00014786', 'GHCND:USW00054743', 'GHCND:USW00054787',
       'GHCND:USW00094728', 'GHCND:USW00094741', 'GHCND:USW00094745',
       'GHCND:USW00094789'],
      dtype='object', length=279)

## Using DataFrame operations to enrich data

In [31]:
import numpy as np

In [32]:
weather = pd.read_csv('data/nyc_weather_2018.csv', parse_dates=['date'])
fb = pd.read_csv('data/fb_2018.csv', index_col='date', parse_dates=True)

Let's find the Z-scores for the volume traded and look at the days where this was more than 3 standard deviations from the mean:

In [33]:
fb.assign(
    abs_z_score_volume=lambda x: x.volume.sub(x.volume.mean()
                        ).div(x.volume.std()).abs()
).query('abs_z_score_volume > 3')

open    high     low   close     volume  abs_z_score_volume
date                                                                     
2018-03-19  177.01  177.17  170.06  172.56   88140060            3.145078
2018-03-20  167.47  170.20  161.95  168.15  129851768            5.315169
2018-03-21  164.80  173.40  163.30  169.39  106598834            4.105413
2018-03-26  160.82  161.10  149.02  160.06  126116634            5.120845
2018-07-26  174.89  180.13  173.75  176.26  169803668            7.393705

`pct_change()` - computes the percentage change from the immediately previous row by default. 

`abs()` - return a Series/DataFrame with absolute numeric value of each element.

`rank()` - compute numerical data ranks (1 through n) along axis. By default, equal values are assigned a rank that is the average of the ranks of those values.

We can use `rank()` and `pct_change()` to see which days had the largest change in volume traded from the day before:

In [34]:
fb.assign(
    volume_pkt_change=fb.volume.pct_change(),
    pct_change_rank=lambda x: x.volume_pkt_change.abs().rank(ascending=False)
).nsmallest(5, 'pct_change_rank')

open    high     low   close     volume  volume_pkt_change  \
date                                                                       
2018-01-12  178.06  181.48  177.40  179.37   77551299           7.087876   
2018-03-19  177.01  177.17  170.06  172.56   88140060           2.611789   
2018-07-26  174.89  180.13  173.75  176.26  169803668           1.628841   
2018-09-21  166.64  167.25  162.81  162.93   45994800           1.428956   
2018-03-26  160.82  161.10  149.02  160.06  126116634           1.352496   

            pct_change_rank  
date                         
2018-01-12              1.0  
2018-03-19              2.0  
2018-07-26              3.0  
2018-09-21              4.0  
2018-03-26              5.0

January 12th was when the news that Facebook changed its news feed product to focus more on content from a users' friends over the brands they follow. Given that Facebook's advertising is a key component of its business ([nearly 89% in 2017](https://www.investopedia.com/ask/answers/120114/how-does-facebook-fb-make-money.asp)), many shares were sold and the price dropped in panic:

In [35]:
fb['2018-01-11':'2018-01-12']

open    high     low   close    volume
date                                                
2018-01-11  188.40  188.40  187.38  187.77   9588587
2018-01-12  178.06  181.48  177.40  179.37  77551299

Throughout 2018, Facebook's stock price never had a low above $215:

In [36]:
(fb > 215).any()

open       True
high       True
low       False
close      True
volume     True
dtype: bool

Facebook's OHLC (open, high, low, and close) prices all had at least one day they were at $215 or less:

In [37]:
(fb > 215).all()

open      False
high      False
low       False
close     False
volume     True
dtype: bool

Sometimes, it's more convenient to work with categories rather than the specific values. A common example is working with ages — most likely, we don't want to look at the data for each age, such as 25 compared to 26; however, we may very well be interested in how the group of individuals aged 25-34 compares to the group of those aged 35-44. 

This is called `binning` or `discretizing` (going from continuous to discrete); we take our data and place the observations into bins (or buckets) matching the range they fall into. By doing so, we can drastically reduce the number of distinct values our data can take on and make it easier to analyze.

When working with volume traded, we may be interested in ranges of volume rather than the exact values. No two days have the same volume traded:

In [38]:
(fb.volume.value_counts() > 1).sum()

0

We can use `pd.cut()` to create 3 bins of even range in volume traded and name them. Then we can work with low, medium, and high volume traded categories:

In [39]:
volume_binned = pd.cut(
    fb.volume, bins=3, labels=['low', 'med', 'high'])
volume_binned.value_counts()

low     240
med       8
high      3
Name: volume, dtype: int64

Let's look at the days with high trading volume:

In [40]:
fb[volume_binned == 'high'].sort_values('volume', ascending=False)

open    high     low   close     volume
date                                                 
2018-07-26  174.89  180.13  173.75  176.26  169803668
2018-03-20  167.47  170.20  161.95  168.15  129851768
2018-03-26  160.82  161.10  149.02  160.06  126116634

July 25th Facebook announced disappointing user growth and the stock tanked in the after hours:

In [41]:
fb['2018-07-25':'2018-07-26']

open    high     low   close     volume
date                                                  
2018-07-25  215.715  218.62  214.27  217.50   64592585
2018-07-26  174.890  180.13  173.75  176.26  169803668

Cambridge Analytica scandal broke on Saturday, March 17th, so we look at the Monday after for the numbers:

In [42]:
fb['2018-03-16':'2018-03-20']

open    high     low   close     volume
date                                                 
2018-03-16  184.49  185.33  183.41  185.09   24403438
2018-03-19  177.01  177.17  170.06  172.56   88140060
2018-03-20  167.47  170.20  161.95  168.15  129851768

Since most days have similar volume, but a few are very large, we have very wide bins. Most of the data is in the low bin. 

If we want each bin to have an equal number of observations, we can split the bins based on evenly-spaced quantiles using the `pd.qcut()` function.

In [43]:
volume_qbinned = pd.qcut(
    fb.volume, q=4, labels=['q1', 'q2', 'q3', 'q4'])
volume_qbinned.value_counts()

q1    63
q2    63
q4    63
q3    62
Name: volume, dtype: int64

The `apply()` method lets us run vectorized operations on entire columns or rows at once. We can apply pretty much any function we can think of as long as those operations are valid on all the columns (or rows) in our data.

In [44]:
central_park_weather = weather.query(
    'station == "GHCND:USW00094728"'
).pivot(index='date', columns='datatype', values='value')

Calculating Z-scores for multiple columns at once

In [45]:
oct_weather_z_scores = central_park_weather\
    .loc['2018-10', ['TMIN', 'TMAX', 'PRCP']]\
    .apply(lambda x: x.sub(x.mean()).div(x.std()))
oct_weather_z_scores.describe().T

count          mean  std       min       25%       50%       75%  \
datatype                                                                     
TMIN       31.0 -1.790682e-16  1.0 -1.339112 -0.751019 -0.474269  1.065152   
TMAX       31.0 -3.010585e-16  1.0 -1.305582 -0.870013 -0.138258  1.011643   
PRCP       31.0  6.088320e-17  1.0 -0.394438 -0.394438 -0.394438 -0.240253   

               max  
datatype            
TMIN      1.843511  
TMAX      1.604016  
PRCP      3.936167

In [46]:
oct_weather_z_scores.query('PRCP > 3').PRCP

date
2018-10-27    3.936167
Name: PRCP, dtype: float64

If we look at the summary statistics for precipitation in October, we can see that this day
had much more precipitation than the rest:

In [47]:
central_park_weather.loc['2018-10', 'PRCP'].describe()

count    31.000000
mean      2.941935
std       7.458542
min       0.000000
25%       0.000000
50%       0.000000
75%       1.150000
max      32.300000
Name: PRCP, dtype: float64

## Window calculations
### Rolling windows

In [48]:
central_park_weather.loc['2018-10'].assign(
    rolling_PRCP=lambda x: x.PRCP.rolling('3D').sum())[['PRCP', 'rolling_PRCP']].head(7).T

date          2018-10-01  2018-10-02  2018-10-03  2018-10-04  2018-10-05  \
datatype                                                                   
PRCP                 0.0        17.5         0.0         1.0         0.0   
rolling_PRCP         0.0        17.5        17.5        18.5         1.0   

date          2018-10-06  2018-10-07  
datatype                              
PRCP                 0.0         0.0  
rolling_PRCP         1.0         0.0

In [49]:
central_park_weather.loc['2018-10']\
    .rolling('3D').mean().head(7).iloc[:, :6]

datatype        AWND      PRCP  SNOW  SNWD       TMAX       TMIN
date                                                            
2018-10-01  0.900000  0.000000   0.0   0.0  24.400000  17.200000
2018-10-02  0.900000  8.750000   0.0   0.0  24.700000  17.750000
2018-10-03  0.966667  5.833333   0.0   0.0  24.233333  17.566667
2018-10-04  0.800000  6.166667   0.0   0.0  24.233333  17.200000
2018-10-05  1.033333  0.333333   0.0   0.0  23.133333  16.300000
2018-10-06  0.833333  0.333333   0.0   0.0  22.033333  16.300000
2018-10-07  1.066667  0.000000   0.0   0.0  22.600000  17.400000

Using `agg()`, we were able to calculate different rolling aggregations for each column

In [50]:
central_park_weather\
    ['2018-10-01':'2018-10-07'].rolling('3D').agg({
    'TMAX': 'max', 'TMIN': 'min',
    'AWND': 'mean', 'PRCP': 'sum'
}).join( # join with original data for comparison
    central_park_weather[['TMAX', 'TMIN', 'AWND', 'PRCP']],
    lsuffix='_rolling'
).sort_index(axis=1) # put rolling calcs next to originals

datatype    AWND  AWND_rolling  PRCP  PRCP_rolling  TMAX  TMAX_rolling  TMIN  \
date                                                                           
2018-10-01   0.9      0.900000   0.0           0.0  24.4          24.4  17.2   
2018-10-02   0.9      0.900000  17.5          17.5  25.0          25.0  18.3   
2018-10-03   1.1      0.966667   0.0          17.5  23.3          25.0  17.2   
2018-10-04   0.4      0.800000   1.0          18.5  24.4          25.0  16.1   
2018-10-05   1.6      1.033333   0.0           1.0  21.7          24.4  15.6   
2018-10-06   0.5      0.833333   0.0           1.0  20.0          24.4  17.2   
2018-10-07   1.1      1.066667   0.0           0.0  26.1          26.1  19.4   

datatype    TMIN_rolling  
date                      
2018-10-01          17.2  
2018-10-02          17.2  
2018-10-03          17.2  
2018-10-04          16.1  
2018-10-05          15.6  
2018-10-06          15.6  
2018-10-07          15.6

### Expanding windows

Rolling calculations (`rolling()`) use a sliding window. Expanding calculations (`expanding()`), however, grow in size. These are equivalent to cumulative aggregations like `cumsum()`; however, we can specify the minimum number of periods required to start calculating (default is 1), and we aren't limited to predefined aggregations. Therefore, while there is no method for the cumulative mean, we can calculate it using `expanding()`. Let's calculate the month-to-date average precipiation:

In [51]:
central_park_weather.loc['2018-06'].assign(
    TOTAL_PRCP=lambda x: x.PRCP.cumsum(),
    AVG_PRCP=lambda x: x.PRCP.expanding().mean()
).head(10)[['PRCP', 'TOTAL_PRCP', 'AVG_PRCP']].T

date        2018-06-01  2018-06-02  2018-06-03  2018-06-04  2018-06-05  \
datatype                                                                 
PRCP               6.9        2.00         6.4        4.10        0.00   
TOTAL_PRCP         6.9        8.90        15.3       19.40       19.40   
AVG_PRCP           6.9        4.45         5.1        4.85        3.88   

date        2018-06-06  2018-06-07  2018-06-08  2018-06-09  2018-06-10  
datatype                                                                
PRCP          0.000000    0.000000       0.000    0.000000        0.30  
TOTAL_PRCP   19.400000   19.400000      19.400   19.400000       19.70  
AVG_PRCP      3.233333    2.771429       2.425    2.155556        1.97

We can also use `agg()` to specify aggregations per column. Note that this works with NumPy functions as well. Here, we join the expanding calculations with the original results for comparison:

In [52]:
central_park_weather\
    ['2018-10-01':'2018-10-07'].expanding().agg({
    'TMAX': np.max, 'TMIN': np.min,
    'AWND': np.mean, 'PRCP': np.sum
}).join(
    central_park_weather[['TMAX', 'TMIN', 'AWND', 'PRCP']],
    lsuffix='_expanding'
).sort_index(axis=1)

datatype    AWND  AWND_expanding  PRCP  PRCP_expanding  TMAX  TMAX_expanding  \
date                                                                           
2018-10-01   0.9        0.900000   0.0             0.0  24.4            24.4   
2018-10-02   0.9        0.900000  17.5            17.5  25.0            25.0   
2018-10-03   1.1        0.966667   0.0            17.5  23.3            25.0   
2018-10-04   0.4        0.825000   1.0            18.5  24.4            25.0   
2018-10-05   1.6        0.980000   0.0            18.5  21.7            25.0   
2018-10-06   0.5        0.900000   0.0            18.5  20.0            25.0   
2018-10-07   1.1        0.928571   0.0            18.5  26.1            26.1   

datatype    TMIN  TMIN_expanding  
date                              
2018-10-01  17.2            17.2  
2018-10-02  18.3            17.2  
2018-10-03  17.2            17.2  
2018-10-04  16.1            16.1  
2018-10-05  15.6            15.6  
2018-10-06  17.2            15.6  
2018-10-07  19.4            15.6

### Exponentially weighted moving windows

In [53]:
central_park_weather.assign(
    AVG=lambda x: x.TMAX.rolling('30D').mean(),
    EWMA=lambda x: x.TMAX.ewm(span=30).mean()
).loc['2018-09-29':'2018-10-08', ['TMAX', 'EWMA', 'AVG']].T

date      2018-09-29  2018-09-30  2018-10-01  2018-10-02  2018-10-03  \
datatype                                                               
TMAX       22.200000   21.100000   24.400000   25.000000   23.300000   
EWMA       24.410887   24.197281   24.210360   24.261304   24.199285   
AVG        24.723333   24.573333   24.533333   24.460000   24.163333   

date      2018-10-04  2018-10-05  2018-10-06  2018-10-07  2018-10-08  
datatype                                                              
TMAX       24.400000   21.700000   20.000000   26.100000   23.300000  
EWMA       24.212234   24.050154   23.788854   23.937960   23.896802  
AVG        23.866667   23.533333   23.070000   23.143333   23.196667

## Pipes 

Pipes are a way to streamline our `pandas` code and make it more readable and flexible.

We can use pipes to apply any function that accepts our data as the first argument and pass in any additional arguments. This makes it easy to chain steps together regardless of whether they are methods or functions:

We can pass any function that will accept the caller of `pipe()` as the first argument:

In [54]:
def get_info(df):
    return '%d rows, %d cols and max closing Z-score: %d' % (*df.shape, df.close.max())

In [55]:
fb.loc['2018-Q1'].apply(lambda x: (x - x.mean())/x.std()).pipe(get_info)

'61 rows, 5 cols and max closing Z-score: 1'

In [56]:
# the pipe is calling pd.DataFrame.rolling(fb, '20D')
fb.pipe(pd.DataFrame.rolling, '20D').mean().equals(fb.rolling('20D').mean())

True

## Aggregating data

In [57]:
fb = pd.read_csv(
    'data/fb_2018.csv', index_col='date', parse_dates=True
    ).assign(trading_volume=lambda x: pd.cut(
    x.volume, bins=3, labels=['low', 'med', 'high']
))
weather = pd.read_csv(
    'data/weather_by_station.csv',
    index_col='date', parse_dates=True
)

In [58]:
weather.head()

datatype            station  value                    station_name
date                                                                         
2018-01-01     PRCP  GHCND:US1CTFR0039    0.0           STAMFORD 4.2 S, CT US
2018-01-01     PRCP  GHCND:US1NJBG0015    0.0  NORTH ARLINGTON 0.7 WNW, NJ US
2018-01-01     SNOW  GHCND:US1NJBG0015    0.0  NORTH ARLINGTON 0.7 WNW, NJ US
2018-01-01     PRCP  GHCND:US1NJBG0017    0.0        GLEN ROCK 0.7 SSE, NJ US
2018-01-01     SNOW  GHCND:US1NJBG0017    0.0        GLEN ROCK 0.7 SSE, NJ US

In [59]:
pd.set_option('display.float_format', lambda x: '%.2f' %x)

We can call `agg()` on the dataframe directly to aggregate its contents into a single series:

In [60]:
fb.agg({
    'open': np.mean, 
    'high': np.max, 
    'low': np.min,
    'close': np.mean, 
    'volume': np.sum
})

open            171.45
high            218.62
low             123.02
close           171.51
volume   6949682394.00
dtype: float64

We can use this to find the total snowfall and precipitation recorded in Central Park in 2018:

In [61]:
weather.query('station == "GHCND:USW00094728"')\
    .pivot(columns='datatype', values='value')\
    [['SNOW', 'PRCP']].sum()

datatype
SNOW   1007.00
PRCP   1665.30
dtype: float64

In [62]:
fb.agg({
    'open': 'mean',
    'high': ['min', 'max'],
    'low': ['min', 'max'],
    'close': 'mean'
})

open   high    low  close
mean 171.45    NaN    NaN 171.51
min     NaN 129.74 123.02    NaN
max     NaN 218.62 214.27    NaN

### Aggregating by group

Often we won't want to aggregate on the entire dataframe, but on groups within it. For this purpose, we can run `groupby()` before the aggregation. If we group by the `trading_volume` column, we will get a row for each of the values it takes on:

In [63]:
fb.groupby('trading_volume').mean()

open   high    low  close       volume
trading_volume                                         
low            171.36 173.46 169.31 171.43  24547207.71
med            175.82 179.42 172.11 175.14  79072559.12
high           167.73 170.48 161.57 168.16 141924023.33

In [64]:
fb.groupby('trading_volume')['close'].agg(['min', 'max', 'mean'])

min    max   mean
trading_volume                     
low            124.06 214.67 171.43
med            152.22 217.50 175.14
high           160.06 176.26 168.16

We can still provide a dictionary specifying the aggregations to perform, but passing a list for a column will result in a hierarchical index for the columns:

In [65]:
fb_agg = fb.groupby('trading_volume').agg({
    'open': 'mean', 
    'high': ['min', 'max'],
    'low': ['min', 'max'],
    'close': 'mean'
})
fb_agg

open   high           low         close
                 mean    min    max    min    max   mean
trading_volume                                          
low            171.36 129.74 216.20 123.02 212.60 171.43
med            175.82 162.85 218.62 150.75 214.27 175.14
high           167.73 161.10 180.13 149.02 173.75 168.16

The columns are stored in a MultiIndex object:

In [66]:
fb_agg.columns

MultiIndex([( 'open', 'mean'),
            ( 'high',  'min'),
            ( 'high',  'max'),
            (  'low',  'min'),
            (  'low',  'max'),
            ('close', 'mean')],
           )

If we want to select the minimum low price for the medium volume traded bucket, we need to use:

In [67]:
fb_agg.loc['med', 'low']['min']

150.75

We can use a list comprehension to remove this hierarchy and instead have our column names in the form of `<column>_<agg>`. At each iteration, we will get a tuple of the levels from the MultiIndex object, which we can combine into a single string to remove the hierarchy:

In [68]:
fb_agg.columns = ['_'.join(col_agg)
                 for col_agg in fb_agg.columns]
fb_agg.head()

open_mean  high_min  high_max  low_min  low_max  close_mean
trading_volume                                                             
low                171.36    129.74    216.20   123.02   212.60      171.43
med                175.82    162.85    218.62   150.75   214.27      175.14
high               167.73    161.10    180.13   149.02   173.75      168.16

In [69]:
fb_agg.columns

Index(['open_mean', 'high_min', 'high_max', 'low_min', 'low_max',
       'close_mean'],
      dtype='object')

We can group on values in the index if we tell `groupby()`, which level to use:

In [70]:
weather.loc['2018-10'].query('datatype == "PRCP"')\
    .groupby(level=0).mean().head().squeeze()

date
2018-10-01    0.01
2018-10-02    2.23
2018-10-03   19.69
2018-10-04    0.32
2018-10-05    0.97
Name: value, dtype: float64

We can also group by many categories at once. We can also create a `Grouper` object, which can also roll up the datetimes in the index. Here, we find the quarterly total precipitation per station:

In [71]:
weather.query('datatype == "PRCP"').groupby(
    ['station_name', pd.Grouper(freq='Q')]
).sum().unstack().sample(5, random_state=0)

value                        \
date                                 2018-03-31 2018-06-30 2018-09-30   
station_name                                                            
RIVER EDGE 0.4 NNE, NJ US                299.60     257.80     563.70   
CANOE BROOK, NJ US                       348.60     302.50     458.80   
PARSIPPANY TROY HILLS TWP 1.3, NJ US     383.60     324.10        NaN   
AMITYVILLE 0.6 NNE, NY US                385.70     254.40     471.70   
ELMSFORD 0.8 SSW, NY US                  285.30     214.00     376.40   

                                                 
date                                 2018-12-31  
station_name                                     
RIVER EDGE 0.4 NNE, NJ US                330.40  
CANOE BROOK, NJ US                       425.30  
PARSIPPANY TROY HILLS TWP 1.3, NJ US        NaN  
AMITYVILLE 0.6 NNE, NY US                470.20  
ELMSFORD 0.8 SSW, NY US                  403.60

Note that we can use `filter()` to exclude some groups from aggregation. Here, we only keep groups with names ending in "NY US" in the group's `name` attribute, which is the station name in this case:

In [72]:
weather.groupby('station_name').filter( # station names with "NY US" in them
    lambda x: x.name.endswith('NY US')
).query('datatype == "SNOW"').groupby('station_name').sum().squeeze() # aggregate and make a series (squeeze)

station_name
ALBERTSON 0.2 SSE, NY US           1087.00
AMITYVILLE 0.1 WSW, NY US           434.00
AMITYVILLE 0.6 NNE, NY US          1072.00
ARMONK 0.3 SE, NY US               1504.00
BROOKLYN 3.1 NW, NY US              305.00
CENTERPORT 0.9 SW, NY US            799.00
CENTERPORT, NY US                  1333.00
ELMSFORD 0.8 SSW, NY US             863.00
FLORAL PARK 0.4 W, NY US           1015.00
HICKSVILLE 1.3 ENE, NY US           716.00
JACKSON HEIGHTS 0.3 WSW, NY US      107.00
JFK INTERNATIONAL AIRPORT, NY US    855.00
LA GUARDIA AIRPORT, NY US           892.00
LOCUST VALLEY 0.3 E, NY US            0.00
LYNBROOK 0.3 NW, NY US              325.00
MASSAPEQUA 0.9 SSW, NY US            41.00
MIDDLE VILLAGE 0.5 SW, NY US       1249.00
NEW HYDE PARK 1.6 NE, NY US           0.00
NEW YORK 8.8 N, NY US                 0.00
NORTH WANTAGH 0.4 WSW, NY US        471.00
NY CITY CENTRAL PARK, NY US        1007.00
PLAINEDGE 0.4 WSW, NY US            610.00
PLAINVIEW 0.4 ENE, NY US           1360.0

Let's see which months have the most precipitation. First, we need to group by day and average the precipitation across the stations. Then, we can group by month and sum the resulting precipitation. Finally, we will use nlargest() to get the five months with the most precipitation:

In [73]:
weather.query('datatype == "PRCP"') \
    .groupby(level=0).mean() \
    .groupby(pd.Grouper(freq='M')).sum().value.nlargest()

date
2018-11-30   210.59
2018-09-30   193.09
2018-08-31   192.45
2018-07-31   160.98
2018-02-28   158.11
Name: value, dtype: float64

### Pivot tables and crosstabs

With `pivot_table()`, we get the mean by default. In its simplest form, we provide a column to place along the columns:

In [74]:
fb.pivot_table(columns='trading_volume')

trading_volume         low         med         high
close               171.43      175.14       168.16
high                173.46      179.42       170.48
low                 169.31      172.11       161.57
open                171.36      175.82       167.73
volume         24547207.71 79072559.12 141924023.33

In [75]:
weather.reset_index().pivot_table(
    index=['date', 'station', 'station_name'],
    columns='datatype',
    values='value',
    aggfunc='median'
).reset_index().tail()

datatype       date            station                         station_name  \
28740    2018-12-31  GHCND:USW00054787  FARMINGDALE REPUBLIC AIRPORT, NY US   
28741    2018-12-31  GHCND:USW00094728          NY CITY CENTRAL PARK, NY US   
28742    2018-12-31  GHCND:USW00094741             TETERBORO AIRPORT, NJ US   
28743    2018-12-31  GHCND:USW00094745        WESTCHESTER CO AIRPORT, NY US   
28744    2018-12-31  GHCND:USW00094789     JFK INTERNATIONAL AIRPORT, NY US   

datatype  AWND  DAPR  MDPR    PGTM  PRCP  SNOW  SNWD  ...  WSF5  WT01  WT02  \
28740     5.00   NaN   NaN 2052.00 28.70   NaN   NaN  ... 15.70   NaN   NaN   
28741      NaN   NaN   NaN     NaN 25.90  0.00  0.00  ...   NaN  1.00   NaN   
28742     1.70   NaN   NaN 1954.00 29.20   NaN   NaN  ...  8.90   NaN   NaN   
28743     2.70   NaN   NaN 2212.00 24.40   NaN   NaN  ... 11.20   NaN   NaN   
28744     4.10   NaN   NaN     NaN 31.20  0.00  0.00  ... 12.50  1.00  1.00   

datatype  WT03  WT04  WT05  WT06  WT08  WT09  WT11  
28740      NaN   NaN   NaN   NaN   NaN   NaN   NaN  
28741      NaN   NaN   NaN   NaN   NaN   NaN   NaN  
28742      NaN   NaN   NaN   NaN   NaN   NaN   NaN  
28743      NaN   NaN   NaN   NaN   NaN   NaN   NaN  
28744      NaN   NaN   NaN   NaN   NaN   NaN   NaN  

[5 rows x 30 columns]

We can use the `pd.crosstab()` function to create a frequency table. If we want to see how many low-, medium-, and high-volume trading days Facebook stock had each month, we can use a crosstab. By default, the
values in the cells will be the count:

In [76]:
pd.crosstab(
    index=fb.trading_volume, columns=fb.index.month,
    colnames=['month'] # name the columns index
)

month           1   2   3   4   5   6   7   8   9   10  11  12
trading_volume                                                
low             20  19  15  20  22  21  18  23  19  23  21  19
med              1   0   4   1   0   0   2   0   0   0   0   0
high             0   0   2   0   0   0   1   0   0   0   0   0

We can normalize with the row or column totals with the `normalize` parameter. This shows percentage of the total:

In [77]:
pd.crosstab(
    index=fb.trading_volume,
    columns=fb.index.month,
    colnames=['month'],
    normalize='columns'
)

month            1    2    3    4    5    6    7    8    9    10   11   12
trading_volume                                                            
low            0.95 1.00 0.71 0.95 1.00 1.00 0.86 1.00 1.00 1.00 1.00 1.00
med            0.05 0.00 0.19 0.05 0.00 0.00 0.10 0.00 0.00 0.00 0.00 0.00
high           0.00 0.00 0.10 0.00 0.00 0.00 0.05 0.00 0.00 0.00 0.00 0.00

To change the aggregation function, we can provide an argument to values and then specify aggfunc.

We will get the average closing price per month, per volume traded bin, with null values when that combination wasn't present in the data:

In [78]:
pd.crosstab(
    index=fb.trading_volume, 
    columns=fb.index.month,
    colnames=['month'], 
    values=fb.close, 
    aggfunc=np.mean
)

month              1      2      3      4      5      6      7      8      9   \
trading_volume                                                                  
low            185.24 180.27 177.07 163.29 182.93 195.27 201.92 177.49 164.38   
med            179.37    NaN 164.76 174.16    NaN    NaN 194.28    NaN    NaN   
high              NaN    NaN 164.11    NaN    NaN    NaN 176.26    NaN    NaN   

month              10     11     12  
trading_volume                       
low            154.19 141.64 137.16  
med               NaN    NaN    NaN  
high              NaN    NaN    NaN

We can also get row and column subtotals with the `margins` parameter. Let's count the number of times each station recorded snow per month and include the subtotals:

In [79]:
snow_data = weather.query('datatype == "SNOW"')
pd.crosstab(
    index=snow_data.station_name,
    columns=snow_data.index.month,
    colnames=['month'],
    values=snow_data.value,
    aggfunc=lambda x: (x > 0).sum(),
    margins=True, # show row and column subtotals
    margins_name='total observation of snow' # subtotals
)

month                              1     2      3     4    5    6    7    8  \
station_name                                                                  
ALBERTSON 0.2 SSE, NY US        3.00  1.00   3.00  1.00 0.00 0.00 0.00 0.00   
AMITYVILLE 0.1 WSW, NY US       1.00  0.00   1.00  1.00 0.00 0.00 0.00 0.00   
AMITYVILLE 0.6 NNE, NY US       3.00  1.00   3.00  1.00 0.00 0.00 0.00 0.00   
ARMONK 0.3 SE, NY US            6.00  4.00   6.00  3.00 0.00 0.00 0.00 0.00   
BLOOMINGDALE 0.7 SSE, NJ US     2.00  1.00   3.00  1.00 0.00 0.00 0.00 0.00   
...                              ...   ...    ...   ...  ...  ...  ...  ...   
WESTFIELD 0.6 NE, NJ US         3.00  0.00   4.00  1.00 0.00  NaN 0.00 0.00   
WOODBRIDGE TWP 1.1 ESE, NJ US   4.00  1.00   3.00  2.00 0.00 0.00 0.00 0.00   
WOODBRIDGE TWP 1.1 NNE, NJ US   2.00  1.00   3.00  0.00 0.00 0.00 0.00 0.00   
WOODBRIDGE TWP 3.0 NNW, NJ US    NaN  0.00   0.00   NaN  NaN 0.00  NaN  NaN   
total observation of snow     190.00 97.00 237.00 81.00 0.00 0.00 0.00 0.00   

month                            9   10    11    12  total observation of snow  
station_name                                                                    
ALBERTSON 0.2 SSE, NY US      0.00 0.00  1.00  0.00                          9  
AMITYVILLE 0.1 WSW, NY US     0.00 0.00  0.00  0.00                          3  
AMITYVILLE 0.6 NNE, NY US     0.00 0.00  0.00  0.00                          8  
ARMONK 0.3 SE, NY US          0.00 0.00  1.00  3.00                         23  
BLOOMINGDALE 0.7 SSE, NJ US   0.00 0.00  0.00  1.00                          8  
...                            ...  ...   ...   ...                        ...  
WESTFIELD 0.6 NE, NJ US       0.00  NaN  1.00   NaN                          9  
WOODBRIDGE TWP 1.1 ESE, NJ US 0.00 0.00  1.00  0.00                         11  
WOODBRIDGE TWP 1.1 NNE, NJ US 0.00 0.00  1.00  0.00                          7  
WOODBRIDGE TWP 3.0 NNW, NJ US  NaN 0.00  0.00   NaN                          0  
total observation of snow     0.00 0.00 49.00 13.00                        667  

[99 rows x 13 columns]

# Working with time series data

In [80]:
fb = pd.read_csv(
    'data/fb_2018.csv',
    index_col='date',
    parse_dates=True
).assign(
    trading_volume=lambda x: pd.cut(
        x.volume, bins=3, labels=['low', 'med', 'high']
))

## Time-based selection and filtering

In [81]:
fb.loc['2018-04'].sample(3)

open   high    low  close    volume trading_volume
date                                                           
2018-04-17 165.83 169.00 165.66 168.66  22743029            low
2018-04-06 157.73 161.42 156.81 157.20  41644812            low
2018-04-04 152.03 155.56 150.51 155.10  49885584            low

In [82]:
fb.loc['2018-q1'].equals(fb['2018-01':'2018-03'])

True

The `first()` method will give us a specified length of time from the beginning of the time series. Here, we ask for a week. January 1, 2018 was a holiday—meaning the market was closed. It was also a Monday, so the week here is only four days:

In [83]:
fb.first('1W')

open   high    low  close    volume trading_volume
date                                                           
2018-01-02 177.68 181.58 177.55 181.42  18151903            low
2018-01-03 181.88 184.78 181.33 184.67  16886563            low
2018-01-04 184.90 186.21 184.10 184.33  13880896            low
2018-01-05 185.59 186.90 184.93 186.85  13574535            low

The `last()` method will take from the end. Since December 31, 2018 was a Monday, the last week only consists of one day:

In [84]:
fb.last('1W')

open   high    low  close    volume trading_volume
date                                                           
2018-12-31 134.45 134.64 129.95 131.09  24625308            low

Suppose that we reindexed the Facebook stock data to include all dates for 2018. We would have null entries for January 1st and any other days the market was closed:

In [85]:
fb_reindexed = fb.reindex(
    pd.date_range('2018-01-01', '2018-12-31', freq='D'))

We can use the `first_valid_index()` and the `last_valid_index()` methods to obtain the index of the first/last non-null entry in our data:

In [86]:
fb_reindexed.loc['2018-q1'].first_valid_index()

Timestamp('2018-01-02 00:00:00', freq='D')

In [87]:
fb_reindexed.loc['2018-q1'].last_valid_index()

Timestamp('2018-03-29 00:00:00', freq='D')

We can use `asof()` to find the last non-null data before the point we are looking for. If we ask for March 31st, we will get the data from the index we got from `fb_reindexed.loc['2018-Q1'].last_valid_index()`, which was March 29th. Note that this works regardless of whether we reindexed:

In [88]:
fb_reindexed.asof('2018-03-31')

open                  155.15
high                  161.42
low                   154.14
close                 159.79
volume           59434293.00
trading_volume           low
Name: 2018-03-31 00:00:00, dtype: object

In [89]:
stock_data_per_minute = pd.read_csv(
    'data/fb_week_of_may_20_per_minute.csv',
    index_col='date',
    parse_dates=True,
    date_parser=lambda x:
        pd.to_datetime(x, format='%Y-%m-%d %H-%M')
)
stock_data_per_minute.head()

open   high    low  close    volume
date                                                     
2019-05-20 09:30:00 181.62 181.62 181.62 181.62 159049.00
2019-05-20 09:31:00 182.61 182.61 182.61 182.61 468017.00
2019-05-20 09:32:00 182.75 182.75 182.75 182.75  97258.00
2019-05-20 09:33:00 182.95 182.95 182.95 182.95  43961.00
2019-05-20 09:34:00 183.06 183.06 183.06 183.06  79562.00

We can use a `Grouper` object to roll up our data to the daily level along with `first` and `last` for open/close and `min` and `max` for high/low per day:

In [90]:
stock_data_per_minute.groupby(pd.Grouper(freq='1D')).agg({
    'open': 'first',
    'high': 'max',
    'low': 'min',
    'close': 'last',
    'volume': 'sum'
})

open   high    low  close      volume
date                                              
2019-05-20 181.62 184.18 181.62 182.72 10044838.00
2019-05-21 184.53 185.58 183.97 184.82  7198405.00
2019-05-22 184.81 186.56 184.01 185.32  8412433.00
2019-05-23 182.50 183.73 179.76 180.87 12479171.00
2019-05-24 182.33 183.52 181.04 181.06  7686030.00

The `at_time()` method allows us to pull out all datetimes that match a certain time. Here, we can grab all the rows from the time the stock market opens (9:30 AM):

In [91]:
stock_data_per_minute.at_time('9:30')

open   high    low  close    volume
date                                                     
2019-05-20 09:30:00 181.62 181.62 181.62 181.62 159049.00
2019-05-21 09:30:00 184.53 184.53 184.53 184.53  58171.00
2019-05-22 09:30:00 184.81 184.81 184.81 184.81  41585.00
2019-05-23 09:30:00 182.50 182.50 182.50 182.50 121930.00
2019-05-24 09:30:00 182.33 182.33 182.33 182.33  52681.00

We can use the `between_time()` method for similar purpose:

In [92]:
stock_data_per_minute.between_time('15:59', '16:00')

open   high    low  close     volume
date                                                      
2019-05-20 15:59:00 182.91 182.91 182.91 182.91  134569.00
2019-05-20 16:00:00 182.72 182.72 182.72 182.72 1113672.00
2019-05-21 15:59:00 184.84 184.84 184.84 184.84   61606.00
2019-05-21 16:00:00 184.82 184.82 184.82 184.82  801080.00
2019-05-22 15:59:00 185.29 185.29 185.29 185.29   96099.00
2019-05-22 16:00:00 185.32 185.32 185.32 185.32 1220993.00
2019-05-23 15:59:00 180.72 180.72 180.72 180.72  109648.00
2019-05-23 16:00:00 180.87 180.87 180.87 180.87 1329217.00
2019-05-24 15:59:00 181.07 181.07 181.07 181.07   52994.00
2019-05-24 16:00:00 181.06 181.06 181.06 181.06  764906.00

Let's see if, on average, more shares were traded within the first 30 minutes of trading or in the last 30 minutes for the week in question. We can combine `between_time()` with `groupby()` to answer this question.
In addition, we need to use `filter()` to exclude groups from the aggregation.

In [93]:
shares_traded_in_first_30_min = stock_data_per_minute\
    .between_time('9:30', '10:00')\
    .groupby(pd.Grouper(freq='1D'))\
    .filter(lambda x: (x.volume > 0).all())\
    .volume.mean()

In [94]:
shares_traded_in_last_30_min = stock_data_per_minute\
    .between_time('15:30', '16:00')\
    .groupby(pd.Grouper(freq='1D'))\
    .filter(lambda x: (x.volume > 0).all())\
    .volume.mean()

In [95]:
shares_traded_in_first_30_min - shares_traded_in_last_30_min

18592.967741935485

In cases where time doesn't matter, we can normalize the times to midnight:

In [96]:
pd.DataFrame(
    dict(before=stock_data_per_minute.index, after=stock_data_per_minute.index.normalize())
).head()

before      after
0 2019-05-20 09:30:00 2019-05-20
1 2019-05-20 09:31:00 2019-05-20
2 2019-05-20 09:32:00 2019-05-20
3 2019-05-20 09:33:00 2019-05-20
4 2019-05-20 09:34:00 2019-05-20

## Shifting for lagged data

We can use `shift()` to create lagged data. By default, the shift will be one period. For example, we can use `shift()` to create a new column that indicates the previous day's closing price.

In [97]:
fb.assign(
    prior_close=lambda x: x.close.shift(),
    after_hours_change_in_price=lambda x: x.open - x.prior_close,
    abs_change=lambda x: x.after_hours_change_in_price.abs()
).nlargest(5, 'abs_change')

open   high    low  close     volume trading_volume  prior_close  \
date                                                                            
2018-07-26 174.89 180.13 173.75 176.26  169803668           high       217.50   
2018-04-26 173.22 176.27 170.80 174.16   77556934            med       159.69   
2018-01-12 178.06 181.48 177.40 179.37   77551299            med       187.77   
2018-10-31 155.00 156.40 148.96 151.79   60101251            low       146.22   
2018-03-19 177.01 177.17 170.06 172.56   88140060            med       185.09   

            after_hours_change_in_price  abs_change  
date                                                 
2018-07-26                       -42.61       42.61  
2018-04-26                        13.53       13.53  
2018-01-12                        -9.71        9.71  
2018-10-31                         8.78        8.78  
2018-03-19                        -8.08        8.08

## Differensed data

Often we are interested in how the values change from one time period to the next. For this, pandas has the `diff()` method. By default, this will calculate the change from time period t-1 to time period t:

In [98]:
(fb.drop(columns='trading_volume') 
     - fb.drop(columns='trading_volume').shift()
).equals(
    fb.drop(columns='trading_volume').diff())

True

We can use `diff()` to easily calculate the day-over-day change in the Facebook stock data

In [99]:
fb.drop(columns='trading_volume').diff().head()

open  high  low  close      volume
date                                          
2018-01-02   NaN   NaN  NaN    NaN         NaN
2018-01-03  4.20  3.20 3.78   3.25 -1265340.00
2018-01-04  3.02  1.43 2.77  -0.34 -3005667.00
2018-01-05  0.69  0.69 0.83   2.52  -306361.00
2018-01-08  1.61  2.00 1.40   1.43  4420191.00

## Resampling

Sometimes, the data is at a granularity that isn't conducive to our analysis. Consider the case where we have data per minute for the full year of 2018. The level of granularity and nature of the data may render plotting useless. Therefore, we will need to aggregate the data to a less granular frequency.

In such case we can use `resample()` method to aggregate our time series data to a different granularity.

In [100]:
stock_data_per_minute.resample('1D').agg({
    'open': 'first',
    'high': 'max',
    'low': 'min',
    'close': 'last',
    'volume': 'sum'
})

open   high    low  close      volume
date                                              
2019-05-20 181.62 184.18 181.62 182.72 10044838.00
2019-05-21 184.53 185.58 183.97 184.82  7198405.00
2019-05-22 184.81 186.56 184.01 185.32  8412433.00
2019-05-23 182.50 183.73 179.76 180.87 12479171.00
2019-05-24 182.33 183.52 181.04 181.06  7686030.00

In [101]:
fb.resample('Q').mean()

open   high    low  close      volume
date                                              
2018-03-31 179.47 181.79 177.04 179.55 32926396.70
2018-06-30 180.37 182.28 178.60 180.70 24055317.75
2018-09-30 180.81 182.89 178.96 181.03 27019824.76
2018-12-31 145.27 147.62 142.72 144.87 26974331.73

We can also use `apply()`. Here, we show the quarterly change from start to end:

In [102]:
fb.drop(columns='trading_volume').resample('Q').apply(
    lambda x: x.last('1D').values - x.first('1D').values)

open   high    low  close    volume
date                                            
2018-03-31 -22.53 -20.16 -23.41 -21.63  41282390
2018-06-30  39.51  38.40  39.84  38.93 -20984389
2018-09-30 -25.04 -28.66 -29.66 -32.90  20304060
2018-12-31 -28.58 -31.24 -31.31 -31.35  -1782369

In [103]:
melted_stock_data = pd.read_csv(
    'data/melted_stock_data.csv',
    index_col='date', parse_dates=True
)
melted_stock_data.head()

price
date                      
2019-05-20 09:30:00 181.62
2019-05-20 09:31:00 182.61
2019-05-20 09:32:00 182.75
2019-05-20 09:33:00 182.95
2019-05-20 09:34:00 183.06

We can use the `ohlc()` method after resampling to recover the OHLC columns:

In [104]:
melted_stock_data.resample('1D').ohlc()['price']

open   high    low  close
date                                  
2019-05-20 181.62 184.18 181.62 182.72
2019-05-21 184.53 185.58 183.97 184.82
2019-05-22 184.81 186.56 184.01 185.32
2019-05-23 182.50 183.73 179.76 180.87
2019-05-24 182.33 183.52 181.04 181.06

Alternatively, we can upsample to increase the granularity. Note this will introduce `NaN` values. We can even call
`asfreq()` after to not aggregate the result

In [105]:
fb.resample('6H').asfreq().head()

open   high    low  close      volume trading_volume
date                                                                      
2018-01-02 00:00:00 177.68 181.58 177.55 181.42 18151903.00            low
2018-01-02 06:00:00    NaN    NaN    NaN    NaN         NaN            NaN
2018-01-02 12:00:00    NaN    NaN    NaN    NaN         NaN            NaN
2018-01-02 18:00:00    NaN    NaN    NaN    NaN         NaN            NaN
2018-01-03 00:00:00 181.88 184.78 181.33 184.67 16886563.00            low

There are many ways to handle these `NaN` values. We can forward-fill with `pad()`:

In [106]:
fb.resample('6H').pad().head()

open   high    low  close    volume trading_volume
date                                                                    
2018-01-02 00:00:00 177.68 181.58 177.55 181.42  18151903            low
2018-01-02 06:00:00 177.68 181.58 177.55 181.42  18151903            low
2018-01-02 12:00:00 177.68 181.58 177.55 181.42  18151903            low
2018-01-02 18:00:00 177.68 181.58 177.55 181.42  18151903            low
2018-01-03 00:00:00 181.88 184.78 181.33 184.67  16886563            low

We can specify a specific value or a method with `fillna()`:

We can use `asfreq()` and `assign()` to specify the action per column:

In [107]:
fb.resample('6H').asfreq().assign(
    volume=lambda x: x.volume.fillna(0), # put 0 when market is closed
    close=lambda x: x.close.fillna(method='ffill'), # carry forward
    # take the closing price if these aren't available
    open=lambda x: x.open.combine_first(x.close),
    high=lambda x: x.high.combine_first(x.close),
    low=lambda x: x.low.combine_first(x.close)
).head()

open   high    low  close      volume trading_volume
date                                                                      
2018-01-02 00:00:00 177.68 181.58 177.55 181.42 18151903.00            low
2018-01-02 06:00:00 181.42 181.42 181.42 181.42        0.00            NaN
2018-01-02 12:00:00 181.42 181.42 181.42 181.42        0.00            NaN
2018-01-02 18:00:00 181.42 181.42 181.42 181.42        0.00            NaN
2018-01-03 00:00:00 181.88 184.78 181.33 184.67 16886563.00            low

## Merging time series

In [108]:
import sqlite3

with sqlite3.connect('data/stocks.db') as connection:
    fb_prices = pd.read_sql(
        'SELECT * FROM fb_prices', connection,
        index_col='date', parse_dates=['date']
    )
    aapl_prices = pd.read_sql(
        'SELECT * FROM aapl_prices', connection,
        index_col='date', parse_dates=['date']
    )

The Facebook prices are at the minute granularity:

In [109]:
fb_prices.index.second.unique()

Int64Index([0], dtype='int64', name='date')

However, the Apple prices have information for the second:

In [110]:
aapl_prices.index.second.unique()

Int64Index([ 0, 52, 36, 34, 55, 35,  7, 12, 59, 17,  5, 20, 26, 23, 54, 49, 19,
            53, 11, 22, 13, 21, 10, 46, 42, 38, 33, 18, 16,  9, 56, 39,  2, 50,
            31, 58, 48, 24, 29,  6, 47, 51, 40,  3, 15, 14, 25,  4, 43,  8, 32,
            27, 30, 45,  1, 44, 57, 41, 37, 28],
           dtype='int64', name='date')

If we use `merge()` or `join()`, we will only have values for both Apple and Facebook when the Apple price was at the top of the minute. Instead, to try and line these up, we can perform an *as of merge*. In order to handle the mismatch, we will specify to merge with the nearest minute (*direction='nearest'*) and require that a match can only occur between times that are within 30 seconds of each other (*tolerance*). This will place the Apple data with the minute that it is closest to, so 9:31:52 will go with 9:32 and 9:37:07 will go with 9:37. Since the times are on the index, we pass in left_index and right_index, just like we did with `merge()`:

In [111]:
pd.merge_asof(
    fb_prices, aapl_prices,
    left_index=True, right_index=True,
    # merge with nearest minute
    direction='nearest',
    tolerance=pd.Timedelta(30, unit='s')
).head()

FB   AAPL
date                             
2019-05-20 09:30:00 181.62 183.52
2019-05-20 09:31:00 182.61    NaN
2019-05-20 09:32:00 182.75 182.87
2019-05-20 09:33:00 182.95 182.50
2019-05-20 09:34:00 183.06 182.11

This is similar to a left join; however, we are more lenient when matching the keys. If we don't want the behavior of a left join, we can use the pd.merge_ordered() function instead. This will allow us to specify our join type, which will be 'outer' by default. This strategy will give us null values whenever the times don't match exactly, but it will at least sort them for us:

In [112]:
pd.merge_ordered(
    fb_prices.reset_index(), aapl_prices.reset_index()
).set_index('date').head()

FB   AAPL
date                             
2019-05-20 09:30:00 181.62 183.52
2019-05-20 09:31:00 182.61    NaN
2019-05-20 09:31:52    NaN 182.87
2019-05-20 09:32:00 182.75    NaN
2019-05-20 09:32:36    NaN 182.50

# Exercises

## Exercise 1

With the earthquakes.csv file, select all the earthquakes in Japan with a magnitude of 4.9 or greater using the mb magnitude type.

In [113]:
quakes = pd.read_csv('data/earthquakes.csv')
faang = pd.read_csv('data/faang.csv', index_col='date', parse_dates=True)

In [114]:
quakes.query(
    'parsed_place == "Japan" and magType == "mb" and mag >= 4.9')

mag magType           time                         place  tsunami  \
1563 4.90      mb  1538977532250  293km ESE of Iwo Jima, Japan        0   
2576 5.40      mb  1538697528010    37km E of Tomakomai, Japan        0   
3072 4.90      mb  1538579732490     15km ENE of Hasaki, Japan        0   
3632 4.90      mb  1538450871260    53km ESE of Hitachi, Japan        0   

     parsed_place  
1563        Japan  
2576        Japan  
3072        Japan  
3632        Japan

## Exercise 2

Create bins for each full number of earthquake magnitude (for instance, the first bin is (0, 1], the second is (1, 2], and so on) with the ml magnitude type and count how many are in each bin.

In [115]:
quakes.query(
    'magType == "ml"'
).assign(mag_interval = pd.cut(quakes.mag, bins=range(0, 10))).mag_interval.value_counts()

(1, 2]    3105
(0, 1]    2207
(2, 3]     862
(3, 4]     122
(4, 5]       2
(5, 6]       1
(6, 7]       0
(7, 8]       0
(8, 9]       0
Name: mag_interval, dtype: int64

## Exercise 3 

Using the faang.csv file, group by the ticker and resample to monthly frequency. Make the following aggregations:
- Mean of the opening price
- Maximum of the high price
- Minimum of the low price
- Mean of the closing price
- Sum of the volume traded

In [116]:
faang.groupby(['ticker']).resample('1M').agg({
    'open': 'mean',
    'high': 'max',
    'low': 'min',
    'close': 'mean',
    'volume': 'sum'
})

open    high     low   close        volume
ticker date                                                    
AAPL   2018-01-31   43.51   45.03   41.17   43.50 2638717600.00
       2018-02-28   41.82   45.15   37.56   41.91 3711577200.00
       2018-03-31   43.76   45.88   41.24   43.62 2854910800.00
       2018-04-30   42.44   44.74   40.16   42.46 2664617200.00
       2018-05-31   46.24   47.59   41.32   46.38 2483905200.00
       2018-06-30   47.18   48.55   45.18   47.16 2110498000.00
       2018-07-31   47.55   48.99   45.85   47.58 1574765600.00
       2018-08-31   53.12   57.22   49.33   53.34 2801275600.00
       2018-09-30   55.58   57.42   53.83   55.52 2715888000.00
       2018-10-31   55.30   58.37   51.52   55.21 3158994000.00
       2018-11-30   47.95   55.59   42.56   47.81 3845305600.00
       2018-12-31   41.31   46.24   36.65   41.07 3595690000.00
AMZN   2018-01-31 1301.38 1472.58 1170.51 1309.01   96371200.00
       2018-02-28 1447.11 1528.70 1265.93 1442.36  137784000.00
       2018-03-31 1542.16 1617.54 1365.20 1540.37  130400100.00
       2018-04-30 1475.84 1638.10 1352.88 1468.22  129919600.00
       2018-05-31 1590.47 1635.00 1546.02 1594.90   71615500.00
       2018-06-30 1699.09 1763.10 1635.09 1698.82   85941300.00
       2018-07-31 1786.31 1880.05 1678.06 1784.65   97521100.00
       2018-08-31 1891.96 2025.57 1776.02 1897.85   96575800.00
       2018-09-30 1969.24 2050.50 1865.00 1966.08   94445500.00
       2018-10-31 1799.63 2033.19 1476.36 1782.06  183220800.00
       2018-11-30 1622.32 1784.00 1420.00 1625.48  139290000.00
       2018-12-31 1572.92 1778.34 1307.00 1559.44  154812700.00
FB     2018-01-31  184.58  190.66  175.80  184.96  495655700.00
       2018-02-28  180.72  195.32  167.18  180.27  516251600.00
       2018-03-31  173.45  186.10  149.02  173.49  996201700.00
       2018-04-30  164.16  177.10  150.51  163.81  750072700.00
       2018-05-31  181.91  192.72  170.23  182.93  401144100.00
       2018-06-30  194.97  203.55  186.43  195.27  387265600.00
       2018-07-31  199.33  218.62  166.56  199.97  647030700.00
       2018-08-31  177.60  188.30  170.27  177.49  548832700.00
       2018-09-30  164.23  173.89  158.87  164.38  500468800.00
       2018-10-31  154.87  165.88  139.03  154.19  622446300.00
       2018-11-30  141.76  154.13  126.85  141.64  518151700.00
       2018-12-31  137.53  147.19  123.02  137.16  558786200.00
GOOG   2018-01-31 1127.20 1186.89 1045.23 1130.77   28738400.00
       2018-02-28 1088.63 1174.00  992.56 1088.21   42382000.00
       2018-03-31 1096.11 1177.05  980.64 1091.49   45353300.00
       2018-04-30 1038.42 1094.17  990.37 1035.70   41715900.00
       2018-05-31 1064.02 1110.75 1006.29 1069.28   31849400.00
       2018-06-30 1136.40 1186.29 1096.01 1137.63   32096000.00
       2018-07-31 1183.46 1273.89 1093.80 1187.59   31940100.00
       2018-08-31 1226.16 1256.50 1188.24 1225.67   28808400.00
       2018-09-30 1176.88 1212.99 1146.91 1175.81   28862400.00
       2018-10-31 1116.08 1209.96  995.83 1110.94   48494700.00
       2018-11-30 1054.97 1095.57  996.02 1056.16   36735100.00
       2018-12-31 1042.62 1124.65  970.11 1037.42   40257600.00
NFLX   2018-01-31  231.27  286.81  195.42  232.91  238377600.00
       2018-02-28  270.87  297.36  236.11  271.44  184585800.00
       2018-03-31  312.71  333.98  275.90  312.23  263449400.00
       2018-04-30  309.13  338.82  271.22  307.47  262006000.00
       2018-05-31  329.78  356.10  305.73  331.54  142050800.00
       2018-06-30  384.56  423.21  352.82  384.13  244031800.00
       2018-07-31  380.97  419.77  328.00  381.52  305393800.00
       2018-08-31  345.41  376.81  310.93  346.26  213122300.00
       2018-09-30  363.33  383.20  335.83  362.64  170832100.00
       2018-10-31  340.03  386.80  271.21  335.45  363589800.00
       2018-11-30  290.64  332.05  250.00  290.34  257126400.00
       2018-12-31  266.31  298.72  231.23  265.30  234310000.00

## Exercise 4

Build a crosstab with the earthquake data between the tsunami column and the magType column. Rather than showing the frequency count, show the maximum magnitude that was observed for each combination. Put the magnitude type along the columns.

In [117]:
pd.crosstab(index=quakes.tsunami,
           columns=quakes.magType,
           values=quakes.mag,
           aggfunc='max')

magType   mb  mb_lg   md   mh   ml  ms_20   mw  mwb  mwr  mww
tsunami                                                      
0       5.60   3.50 4.11 1.10 4.20    NaN 3.83 5.80 4.80 6.00
1       6.10    NaN  NaN  NaN 5.10   5.70 4.41  NaN  NaN 7.50

## Exercise 5

Calculate the rolling 60-day aggregations of the OHLC data by ticker for the FAANG data. Use the same aggregations as exercise 3.

In [118]:
faang.groupby(['ticker']).rolling('60D').agg({
    'open': 'mean',
    'high': 'max',
    'low': 'min',
    'close': 'mean',
    'volume': 'sum' 
})

open   high    low  close       volume
ticker date                                               
AAPL   2018-01-02  42.54  43.08  42.31  43.06 102223600.00
       2018-01-03  42.84  43.64  42.31  43.06 220295200.00
       2018-01-04  42.94  43.64  42.31  43.13 310033600.00
       2018-01-05  43.04  43.84  42.31  43.28 404673600.00
       2018-01-08  43.15  43.90  42.31  43.34 486944800.00
...                  ...    ...    ...    ...          ...
NFLX   2018-12-24 283.51 332.05 233.68 281.93 525657600.00
       2018-12-26 281.84 332.05 231.23 280.78 520444300.00
       2018-12-27 281.07 332.05 231.23 280.16 532679500.00
       2018-12-28 279.92 332.05 231.23 279.46 521973500.00
       2018-12-31 278.43 332.05 231.23 277.45 476314900.00

[1255 rows x 5 columns]

## Exercise 6

Create a pivot table of the FAANG data that compares the stocks. Put the ticker in the rows and show the averages of the OHLC and volume traded data.

In [119]:
faang.pivot_table(index='ticker')

close    high     low    open       volume
ticker                                             
AAPL     47.26   47.75   46.80   47.28 136080258.17
AMZN   1641.73 1662.84 1619.84 1644.07   5648994.42
FB      171.51  173.61  169.30  171.47  27658596.81
GOOG   1113.23 1125.78 1101.00 1113.55   1741965.34
NFLX    319.29  325.22  313.19  319.62  11469624.70

## Exercise 7

Calculate the Z-scores for each numeric column of Amazon's data (ticker is AMZN) in Q4 2018 using apply().

In [120]:
faang.loc['2018-q4'].query(
    'ticker == "AMZN"'
).select_dtypes(include='number').apply(lambda x: x.sub(x.mean()).div(x.std()))

high   low  open  close  volume
date                                       
2018-10-01  2.37  2.50  2.34   2.39   -1.63
2018-10-02  2.23  2.25  2.19   2.16   -0.86
2018-10-03  2.06  2.14  2.07   2.03   -0.92
2018-10-04  1.82  1.78  1.85   1.72   -0.13
2018-10-05  1.63  1.55  1.64   1.58   -0.30
...          ...   ...   ...    ...     ...
2018-12-24 -2.16 -2.19 -2.18  -2.23   -0.14
2018-12-26 -1.61 -1.81 -2.03  -1.34    1.12
2018-12-27 -1.64 -1.63 -1.46  -1.40    0.85
2018-12-28 -1.33 -1.23 -1.33  -1.29    0.50
2018-12-31 -1.27 -0.98 -1.08  -1.12   -0.25

[63 rows x 5 columns]

## Exercise 8

Add event descriptions:
1. Create a dataframe with the following three columns: ticker, date, and event. The columns should have the following values:
- ticker: 'FB'
- date: ['2018-07-25', '2018-03-19', '2018-03-20']
- event: ['Disappointing user growth announced after close.', 'Cambridge Analytica story', 'FTC investigation']
2. Set the index to ['date', 'ticker'].
3. Merge this data with the FAANG data using an outer join.

In [121]:
events = pd.DataFrame({
    'ticker': 'FB',
    'date': pd.to_datetime(['2018-07-25', '2018-03-19', '2018-03-20']),
    'event': [
        'Disappointing user growth announced after close.',
        'Cambridge Analytica story',
        'FTC investigation']
}).set_index(['date', 'ticker'])

faang.reset_index().set_index(['date', 'ticker']).join(
    events, how='outer')['event'].value_counts()

Cambridge Analytica story                           1
FTC investigation                                   1
Disappointing user growth announced after close.    1
Name: event, dtype: int64

In [122]:
date = ['2018-07-25', '2018-03-19', '2018-03-20']
ticker = 'FB'
event = ['Disappointing user growth announced after close.', 'Cambridge Analytica story', 'FTC investigation']
dict_to_df = {
    'date': pd.to_datetime(date),
    'ticker': ticker,
    'event': event
}
new_df = pd.DataFrame(dict_to_df).reset_index().set_index(['date', 'ticker'])

faang.reset_index().set_index(['date', 'ticker']).merge(
    new_df, how='outer', indicator=True, left_index=True, right_index=True)['event'].value_counts()

Cambridge Analytica story                           1
FTC investigation                                   1
Disappointing user growth announced after close.    1
Name: event, dtype: int64

In [123]:
faang[(faang.index.isin(['2018-07-25', '2018-03-19', '2018-03-20']) & (faang.ticker == 'FB'))]

ticker   high    low   open  close       volume
date                                                      
2018-03-19     FB 177.17 170.06 177.01 172.56  88140100.00
2018-03-20     FB 170.20 161.95 167.47 168.15 129851800.00
2018-07-25     FB 218.62 214.27 215.72 217.50  58954200.00

## Exercise 9

Use the `transform()` method on the FAANG data to represent all the values in terms of the first date in the data. To do so, divide all the values for each ticker by the values for the first date in the data for that ticker. This is referred to as an index, and the data for the first date is the base [More information](https://ec.europa.eu/eurostat/statistics-explained/index.php/Beginners:Statistical_concept_-_Index_and_base_year). 

When data is in this format, we can easily see growth over time. *Hint*: `transform()` can take a function name.

In [124]:
faang = faang.reset_index().set_index(['ticker', 'date'])
faang_index = (faang / faang.groupby(level='ticker').transform('first'))

# view 3 rows of the result per ticker
faang_index.groupby(level='ticker').agg('head', 3)

high  low  open  close  volume
ticker date                                      
FB     2018-01-02  1.00 1.00  1.00   1.00    1.00
       2018-01-03  1.02 1.02  1.02   1.02    0.93
       2018-01-04  1.03 1.04  1.04   1.02    0.76
AAPL   2018-01-02  1.00 1.00  1.00   1.00    1.00
       2018-01-03  1.01 1.02  1.01   1.00    1.16
       2018-01-04  1.01 1.02  1.01   1.00    0.88
AMZN   2018-01-02  1.00 1.00  1.00   1.00    1.00
       2018-01-03  1.01 1.02  1.01   1.01    1.15
       2018-01-04  1.02 1.03  1.03   1.02    1.12
NFLX   2018-01-02  1.00 1.00  1.00   1.00    1.00
       2018-01-03  1.02 1.03  1.03   1.02    0.78
       2018-01-04  1.03 1.04  1.05   1.02    0.55
GOOG   2018-01-02  1.00 1.00  1.00   1.00    1.00
       2018-01-03  1.02 1.02  1.02   1.02    1.16
       2018-01-04  1.02 1.04  1.04   1.02    0.81

## Exercise 10

The **European Centre for Disease Prevention and Control (ECDC)** provides an open dataset on COVID-19 cases called [daily number of new reported cases of COVID-19 by country worldwide](https://www.ecdc.europa.eu/en/publications-data/download-todays-data-geographicdistribution-covid-19-cases-worldwide). This dataset is updated daily,
but we will use a snapshot that contains data through September 18, 2020. 

Complete the following tasks to practice the skills you've learned up to this point in the book:
1. Prepare the data:
- Read in the data in the `covid19_cases.csv file`.
- Create a `date` column by parsing the `dateRep` column into a datetime.
- Set the `date` column as the index.
- Use the `replace()` method to update all occurrences of `United_States_of_America` and `United_Kingdom` to `USA` and `UK`, respectively.
- Sort the index.
2. For the five countries with the most cases (cumulative), find the day with the largest number of cases.
3. Find the 7-day average change in COVID-19 cases for the last week in the data for the five countries with the most cases.
4. Find the first date that each country other than China had cases.
5. Rank the countries by cumulative cases using percentiles.

In [125]:
# 1. Prepare the data
cases = pd.read_csv('data/covid19_cases.csv')\
    .assign(date=lambda x: pd.to_datetime(x.dateRep, format='%d/%m/%Y'))\
    .set_index('date')\
    .replace('United_States_of_America', 'USA')\
    .replace('United_Kingdom', 'UK')\
    .sort_index()

In [126]:
# 2. For the five countries with the most cases (cumulative), find the day with the largest number of cases.
countries = cases.groupby('countriesAndTerritories').cases.sum().nlargest(5).index
cases[cases.countriesAndTerritories.isin(countries)]\
    .groupby('countriesAndTerritories').cases.idxmax()

countriesAndTerritories
Brazil   2020-07-30
India    2020-09-17
Peru     2020-08-17
Russia   2020-07-18
USA      2020-07-25
Name: cases, dtype: datetime64[ns]

In [127]:
# 3. Find the 7-day average change in COVID-19 cases for the last week in the data 
# for the five countries with the most cases.
cases.groupby(['countriesAndTerritories', pd.Grouper(freq='1D')]).cases.sum()\
        .unstack(0).diff().rolling(7).mean().last('1W')[countries]

countriesAndTerritories      USA   India  Brazil  Russia    Peru
date                                                            
2020-09-14                473.71  181.29   35.29   36.29   73.14
2020-09-15               1513.00 1142.86  697.43   46.29  377.57
2020-09-16               3478.71   59.57 3196.29   61.43  -65.00
2020-09-17              -1047.00  308.43  143.43  810.00  -29.43
2020-09-18                865.71  -18.14 -607.71 -688.43 -227.57
2020-09-19                306.86 -604.71 -560.14   57.29  -41.29

In [128]:
# 4. Find the first date that each country other than China had cases.
cases.reset_index()\
    .pivot(index='date', columns='countriesAndTerritories', values='cases')\
    .drop(columns='China')\
    .fillna(0)\
    .apply(lambda x: x[(x > 0)].idxmin())\
    .sort_values()\
    .rename(lambda x: x.replace('_', ' '))

countriesAndTerritories
Thailand         2020-01-13
Japan            2020-01-15
South Korea      2020-01-20
USA              2020-01-21
Taiwan           2020-01-21
                    ...    
Lesotho          2020-05-15
Uruguay          2020-05-17
Western Sahara   2020-06-20
Mali             2020-07-07
Puerto Rico      2020-09-10
Length: 209, dtype: datetime64[ns]

In [129]:
# 5. Rank the countries by cumulative cases using percentiles.
cases.pivot_table(
    columns='countriesAndTerritories', 
    values='cases', 
    aggfunc='sum')\
    .T.transform('rank', method='max', pct=True)\
    .sort_values(by='cases', ascending=False)\
    .rename(lambda x: x.replace('_', ' '))

cases
countriesAndTerritories           
USA                           1.00
India                         1.00
Brazil                        0.99
Russia                        0.99
Peru                          0.98
...                            ...
Greenland                     0.02
Montserrat                    0.02
Falkland Islands (Malvinas)   0.02
Holy See                      0.01
Anguilla                      0.00

[210 rows x 1 columns]